In [1]:
import os, datetime
import pandas as pd
import numpy as np
import seaborn as sn
import copy
import tqdm
import talib

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
# from torch.utils.data import Dataset, DataLoader

# from sklearn import tree, svm
# from sklearn.pipeline import make_pipeline
# from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

plt.style.use('fivethirtyeight')
plt.rcParams['lines.linewidth'] = .5
plt.rcParams['font.size'] = 10
plt.rcParams['figure.figsize'] = (16, 10)

In [2]:
from lutils.fin.autoencoder import Normalizer

In [3]:
# np.set_printoptions(precision=4, suppress=True)

In [4]:
# %matplotlib inline
%matplotlib qt

In [5]:
# from lutils.fin.data_loader import load, load_ctp

In [6]:
# rbs = (
#     ('2021-04-06', 'SHFE.rb2110'),
#     ('2021-08-10', 'SHFE.rb2201'),
#     ('2021-11-25', 'SHFE.rb2205'),
#     ('2022-03-29', 'SHFE.rb2210'),
#     ('2022-08-30', 'SHFE.rb2301'),
#     ('2022-12-02', 'SHFE.rb2305'),
#     ('2023-04-04', 'SHFE.rb2310'),
# )

In [7]:
# dfs = []
# for i, (d, exchange_symbol) in enumerate(rbs):
#     start_date = d
    
#     exchange, symbol = exchange_symbol.split('.')
#     df = load(exchange, symbol)
#     df.index = df.datetime
#     if i < len(rbs) - 1:
#         end_date = rbs[i+1][0]
#         dfs.append(df[(df.index >= start_date) & (df.index < end_date)])
#     else:
#         dfs.append(df[start_date:])

In [8]:
# dfs[-3]

In [9]:
# df = pd.concat(dfs)

In [13]:
# df

datetime  last_price  volume  \
datetime                                                                    
2021-04-06 09:00:00.500001 2021-04-06 09:00:00.500001      5135.0    4305   
2021-04-06 09:00:01.000000 2021-04-06 09:00:01.000000      5137.0    5696   
2021-04-06 09:00:01.500000 2021-04-06 09:00:01.500000      5139.0    6845   
2021-04-06 09:00:02.000000 2021-04-06 09:00:02.000000      5140.0    8111   
2021-04-06 09:00:02.500000 2021-04-06 09:00:02.500000      5139.0    9110   
...                                               ...         ...     ...   
2023-06-09 22:59:58.500000 2023-06-09 22:59:58.500000      3693.0  672313   
2023-06-09 22:59:59.000000 2023-06-09 22:59:59.000000      3693.0  672370   
2023-06-09 22:59:59.500000 2023-06-09 22:59:59.500000      3693.0  672448   
2023-06-09 23:00:00.000000 2023-06-09 23:00:00.000000      3694.0  672503   
2023-06-09 23:00:00.500000 2023-06-09 23:00:00.500000      3694.0  672503   

                                  amount  open_interest  bid_price1  \
datetime                                                              
2021-04-06 09:00:00.500001  2.210646e+08      1053464.0      5135.0   
2021-04-06 09:00:01.000000  2.925133e+08      1053828.0      5137.0   
2021-04-06 09:00:01.500000  3.515532e+08      1054089.0      5139.0   
2021-04-06 09:00:02.000000  4.165743e+08      1053951.0      5137.0   
2021-04-06 09:00:02.500000  4.678919e+08      1053769.0      5138.0   
...                                  ...            ...         ...   
2023-06-09 22:59:58.500000  2.490649e+10      1908556.0      3692.0   
2023-06-09 22:59:59.000000  2.490860e+10      1908526.0      3692.0   
2023-06-09 22:59:59.500000  2.491148e+10      1908469.0      3693.0   
2023-06-09 23:00:00.000000  2.491351e+10      1908466.0      3693.0   
2023-06-09 23:00:00.500000  2.491351e+10      1908466.0      3693.0   

                            bid_volume1  ask_price1  ask_volume1  
datetime                                                          
2021-04-06 09:00:00.500001           86      5137.0            1  
2021-04-06 09:00:01.000000           19      5139.0           22  
2021-04-06 09:00:01.500000            1      5140.0          523  
2021-04-06 09:00:02.000000            4      5140.0          426  
2021-04-06 09:00:02.500000            6      5139.0            8  
...                                 ...         ...          ...  
2023-06-09 22:59:58.500000          306      3693.0           72  
2023-06-09 22:59:59.000000          300      3693.0           32  
2023-06-09 22:59:59.500000           81      3694.0           16  
2023-06-09 23:00:00.000000           60      3694.0           18  
2023-06-09 23:00:00.500000           60      3694.0           18  

[21676970 rows x 9 columns]

In [14]:
# store = pd.HDFStore('D:/option/SHFE.rb.h5', 'w', complevel=7)
# store.append('rb', df)
# store.flush()
# store.close()

In [2]:
store = pd.HDFStore('D:/option/SHFE.rb.h5', 'r')
df = store['rb']
store.close()

In [3]:
# df.volume = df.volume.diff(1)
# df.amount = df.amount.diff(1)

In [12]:
df[np.isnan(df.volume)].index

DatetimeIndex(['2021-04-06 09:00:00.500001'], dtype='datetime64[ns]', name='datetime', freq=None)

In [13]:
# df = df.drop(df[df.volume < 0].index)
df = df.drop(df[np.isnan(df.volume)].index)

In [17]:
df[df.amount == 0]

datetime  last_price  volume  amount  \
datetime                                                                      
2021-04-06 09:11:18.500 2021-04-06 09:11:18.500      5152.0     0.0     0.0   
2021-04-06 09:12:40.500 2021-04-06 09:12:40.500      5152.0     0.0     0.0   
2021-04-06 09:12:51.500 2021-04-06 09:12:51.500      5151.0     0.0     0.0   
2021-04-06 09:13:00.000 2021-04-06 09:13:00.000      5150.0     0.0     0.0   
2021-04-06 09:13:46.500 2021-04-06 09:13:46.500      5148.0     0.0     0.0   
...                                         ...         ...     ...     ...   
2023-06-09 22:56:35.000 2023-06-09 22:56:35.000      3696.0     0.0     0.0   
2023-06-09 22:56:42.000 2023-06-09 22:56:42.000      3697.0     0.0     0.0   
2023-06-09 22:57:13.500 2023-06-09 22:57:13.500      3697.0     0.0     0.0   
2023-06-09 22:59:17.500 2023-06-09 22:59:17.500      3698.0     0.0     0.0   
2023-06-09 23:00:00.500 2023-06-09 23:00:00.500      3694.0     0.0     0.0   

                         open_interest  bid_price1  bid_volume1  ask_price1  \
datetime                                                                      
2021-04-06 09:11:18.500      1064395.0      5150.0          170      5151.0   
2021-04-06 09:12:40.500      1064766.0      5151.0          124      5152.0   
2021-04-06 09:12:51.500      1065064.0      5151.0           46      5152.0   
2021-04-06 09:13:00.000      1065226.0      5149.0           48      5150.0   
2021-04-06 09:13:46.500      1065217.0      5147.0          203      5148.0   
...                                ...         ...          ...         ...   
2023-06-09 22:56:35.000      1913775.0      3696.0           78      3697.0   
2023-06-09 22:56:42.000      1913662.0      3696.0          181      3697.0   
2023-06-09 22:57:13.500      1913654.0      3696.0          489      3697.0   
2023-06-09 22:59:17.500      1911083.0      3697.0          276      3698.0   
2023-06-09 23:00:00.500      1908466.0      3693.0           60      3694.0   

                         ask_volume1  
datetime                              
2021-04-06 09:11:18.500           62  
2021-04-06 09:12:40.500           88  
2021-04-06 09:12:51.500          248  
2021-04-06 09:13:00.000           37  
2021-04-06 09:13:46.500           98  
...                              ...  
2023-06-09 22:56:35.000          396  
2023-06-09 22:56:42.000          259  
2023-06-09 22:57:13.500          345  
2023-06-09 22:59:17.500           90  
2023-06-09 23:00:00.500           18  

[1227353 rows x 9 columns]

In [21]:
sn.displot(df.volume.values[:20000], kde=True)

In [15]:
store = pd.HDFStore('D:/option/SHFE.rb2.h5', 'w', complevel=7)
store.append('rb', df)
store.flush()
store.close()

In [5]:
def sec_df(df, exchange, symbol, rule='1S'):
    
    between_times = [['09:00', '10:15'], ['10:30', '11:30'], ['13:30', '15:00'], ['21:00', '23:00']]
    
    dfs = []
    with tqdm.tqdm(np.unique(df.index.date)) as bar:
        for d in bar:
            bar.set_postfix({'date': d})
            ddd = df[df.index.date == d]
            if ddd.shape[0] > 1:
                day1S_df = ddd.resample(rule).last()
                day1S_df = day1S_df.ffill()
                for (start, end) in between_times:
                    _df = day1S_df.between_time(start, end)
                    if _df.shape[0] > 1:
                        dfs.append(_df)
            
    _df = pd.concat(dfs)
    store = pd.HDFStore('D:/option/SHFE.rb.%s.h5' % rule, 'w', complevel=7)
    store.append('rb', _df)
    store.flush()
    store.close()

In [7]:
sec_df(df, 'SHFE', 'rb', '2S')

100%|███████████████████████| 534/534 [40:40<00:00,  4.57s/it, date=2023-06-09]


In [8]:
sec_df(df, 'SHFE', 'rb', '3S')

100%|███████████████████████| 534/534 [41:33<00:00,  4.67s/it, date=2023-06-09]


In [9]:
store = pd.HDFStore('D:/option/SHFE.rb.1S.h5', 'r')
df = store['rb']
store.close()

In [11]:
d = df.loc['2021-04-06'].between_time('09:00', '10:15')

In [20]:
d = d.ffill()

In [75]:
macd, macdsignal, macdhist = talib.MACD(d.last_price, fastperiod=11, slowperiod=23, signalperiod=7)

In [88]:
upper, middle, lower = talib.BBANDS(d.last_price, matype=talib.MA_Type.T3)

In [97]:
sn.displot(lower, kde=True)

In [81]:
sn.displot(macd[60:], kde=True)

In [82]:
plt.plot(macd)
plt.plot(macdsignal)
plt.plot(macdhist)
plt.show()

In [83]:
def plot_macd(prices, macd, signal, hist):
    ax1 = plt.subplot2grid((8,1), (0,0), rowspan = 5, colspan = 1)
    ax2 = plt.subplot2grid((8,1), (5,0), rowspan = 3, colspan = 1)

    ax1.plot(prices)
    ax2.plot(macd, color = 'grey', linewidth = 0.5, label = 'MACD')
    ax2.plot(signal, color = 'skyblue', linewidth = 0.5, label = 'SIGNAL')

    for i in range(len(prices)):
        if str(hist[i])[0] == '-':
            ax2.bar(prices.index[i], hist[i], color = '#ef5350')
        else:
            ax2.bar(prices.index[i], hist[i], color = '#26a69a')

    plt.legend(loc = 'lower right')

In [93]:
def plot_boll(prices, upper, middle, lower):
    ax1 = plt.subplot2grid((8,1), (0,0), rowspan = 5, colspan = 1)
    ax2 = plt.subplot2grid((8,1), (5,0), rowspan = 3, colspan = 1)

    ax1.plot(prices)
    ax2.plot(upper, color = 'grey', linewidth = 0.5, label = 'UPPER')
    ax2.plot(lower, color = 'skyblue', linewidth = 0.5, label = 'LOWER')
    ax2.plot(middle, color = '#ef5350', linewidth = 0.5, label = 'MIDDLE')
    
    plt.legend(loc = 'lower right')

In [91]:
start = 100
end= 600

In [85]:
price = d.last_price[start:end]
_macd = macd[start:end]
signal = macdsignal[start:end]
hist = macdhist[start:end]

price.index = pd.RangeIndex(start=0, stop=500, step=1)
_macd.index = pd.RangeIndex(start=0, stop=500, step=1)
signal.index = pd.RangeIndex(start=0, stop=500, step=1)
hist.index = pd.RangeIndex(start=0, stop=500, step=1)

In [92]:
_upper = upper[start:end]
_middle = middle[start:end]
_lower = lower[start:end]

_upper.index = pd.RangeIndex(start=0, stop=500, step=1)
_middle.index = pd.RangeIndex(start=0, stop=500, step=1)
_lower.index = pd.RangeIndex(start=0, stop=500, step=1)

In [87]:
plot_macd(price, _macd, signal, hist)

In [94]:
plot_boll(price, _upper, _middle, _lower)

In [66]:
macd[100:600]

datetime
2021-04-06 09:01:40    0.584607
2021-04-06 09:01:41    0.839882
2021-04-06 09:01:42    1.479274
2021-04-06 09:01:43    1.943584
2021-04-06 09:01:44    2.105831
                         ...   
2021-04-06 09:09:55   -0.123555
2021-04-06 09:09:56   -0.254480
2021-04-06 09:09:57   -0.135798
2021-04-06 09:09:58   -0.061885
2021-04-06 09:09:59   -0.017017
Length: 500, dtype: float64

### 前15分钟    后5分钟 涨跌幅

In [ ]:
SAVE_PATH = 'D:/option/l8'

In [ ]:
def prepare_data(df, exchange, symbol):
    
    between_times = [['09:00', '10:15'], ['10:30', '11:30'], ['13:30', '15:00'], ['21:00', '23:00']]
    start_ix = 900
    next_tick_sec = 300
    
    def _prepare_data(x, window_size):
        n_row = x.shape[0] - window_size + 1
        output = np.lib.stride_tricks.as_strided(x, shape=(n_row, window_size), strides=(x.strides[0],x.strides[0]))
#         return output[:-1], output[-1]
        return output
    
#     dfs_x = []
#     dfs_y = []
#     last_prices = []
#     for d in tqdm.tqdm(np.unique(df.index.date)):
    with tqdm.tqdm(np.unique(df.index.date)) as bar:
        bar.set_description('%s %s' % (exchange, symbol))
        for d in bar:
            ddd = df[df.index.date == d]
            p = os.path.join(SAVE_PATH, '%s.%s.%s.%s.h5' % (exchange, symbol, datetime.datetime.strftime(d, '%Y-%m-%d'), ddd.shape[0]))
            if os.path.exists(p):
                continue
            if ddd.shape[0] > 1:
                day1S_df = ddd.resample('1S').last()
                day1S_df = day1S_df.interpolate()
                day1S_df = day1S_df.dropna()
                
                dfs_x = []
                dfs_y = []
                last_prices = []
                for (start, end) in between_times:
                    # X
                    _df = day1S_df.between_time(start, end)

                    if _df.shape[0] > 100:
                        ix = _df.index.values

                        df_x = _prepare_data(_df.last_price.values, start_ix) # len - 900 + 1
                        df_y = np.roll(df_x, -start_ix, axis=0)[:-start_ix,:next_tick_sec] # len - 900 + 1 - 900
                        df_x = df_x[:-start_ix]
                        
                        df_x = df_x - df_x.mean(axis=1, keepdims=True)

        #                 y = (df_y[:, 0] - df_y[:, -1]) / (df_y.max(axis=1) - df_y.min(axis=1))
                        df_y = (df_y[:, -1] - df_y[:, 0]) / df_y[:, 0]

                        df_y[np.isinf(df_y) | np.isneginf(df_y)] = 0
                        df_y[np.isnan(df_y)] = 0
            #             y[df_y[:, 0] > df_y[:, -1]] = y[df_y[:, 0] > df_y[:, -1]] * -1
            #             if y.sum() > 0 and np.abs(y.mean()) != 1:
                        
#                         print(ix.shape)
#                         print(df_x.shape)
#                         print(df_y.shape)
                    
                        df_x = pd.DataFrame(df_x, index=np.roll(ix, -start_ix)[:-start_ix-start_ix+1]) # 前 start_ix 个数据的结束时间
                        df_y = pd.DataFrame(df_y, index=np.roll(ix, -start_ix-next_tick_sec)[:-start_ix-start_ix+1]) # 预测后 next_tick_sec 结束时间

                        dfs_x.append(df_x) # .iloc[:-start_ix])
                        dfs_y.append(df_y)
                        last_prices.append(_df.last_price)
                
                if len(dfs_x) > 0:
                    df_x = pd.concat(dfs_x)
                    df_y = pd.concat(dfs_y)
                    df_last_price = pd.concat(last_prices)


                    store = pd.HDFStore(p, 'w', complevel=7)

                    store.append('X', df_x)
                    store.append('y', df_y)
                    store.append('last_price', df_last_price)

                    store.flush()
                    store.close()
#                 print(exchange, symbol, d)
            
#     return #dfs_x, dfs_y, last_prices

In [ ]:
def dump_data(aa):
    for f in aa:
        exchange, symbol = f.split('.')
        df = load(exchange, symbol)
        df.index = df.datetime
        prepare_data(df[['last_price', 'volume', 'amount']], exchange, symbol)
        

In [ ]:
aa = [] # 'SHFE.rb2306', 'SHFE.rb2307'
# 'CZCE.MA307', 'DCE.m2303', 'DCE.c2209' 'SHFE.rb2305',  'DCE.a2207', 'DCE.pp2210', , 'DCE.v2206', 'DCE.pp2112'
dump_data(aa)

In [ ]:
for df in dfs:
    exchange = 'SHFE'
    symbol = 'rb'
    prepare_data(df[['last_price', 'volume', 'amount']], exchange, symbol)

In [ ]:
dfs[2]['2022-01-03']

In [ ]:
store = pd.HDFStore(os.path.join(SAVE_PATH, 'SHFE.rb.2021-04-08.41369.h5'))
df_x = store['X']
df_y = store['y']

In [ ]:
# df_x

In [ ]:
def _prepare_data(x, window_size):
    n_row = x.shape[0] - window_size + 1
    output = np.lib.stride_tricks.as_strided(x, shape=(n_row, window_size), strides=(x.strides[0],x.strides[0]))
#         return output[:-1], output[-1]
    return output

In [ ]:
exchange = 'SHFE'
symbol = 'rb2305'

In [ ]:
df = load(exchange, symbol)
df.index = df.datetime

In [ ]:
ddd = df.between_time('09:00', '10:15')

In [ ]:
d = _prepare_data(ddd.last_price.values, 20)

In [ ]:
ddd.shape

In [ ]:
d[0]

In [ ]:
sn.displot(df_x[110], kde=True)

In [ ]:
sn.displot(df_y.values, kde=True)

In [ ]:
df_y.values

### 15分钟 后5分钟 分类

In [ ]:
SAVE_PATH = 'D:/option/l8_5mean_classification' # 

In [ ]:
def prepare_data(df, exchange, symbol):
    
    between_times = [['09:00', '10:15'], ['10:30', '11:30'], ['13:30', '15:00'], ['21:00', '23:00']]
    start_ix = 900
    next_tick_sec = 300
    
    def _prepare_data(x, window_size):
        n_row = x.shape[0] - window_size + 1
        output = np.lib.stride_tricks.as_strided(x, shape=(n_row, window_size), strides=(x.strides[0],x.strides[0]))
#         return output[:-1], output[-1]
        return output
    
#     dfs_x = []
#     dfs_y = []
#     last_prices = []
#     for d in tqdm.tqdm(np.unique(df.index.date)):
    with tqdm.tqdm(np.unique(df.index.date)) as bar:
        bar.set_description('%s %s' % (exchange, symbol))
        for d in bar:
            ddd = df[df.index.date == d]
            p = os.path.join(SAVE_PATH, '%s.%s.%s.%s.h5' % (exchange, symbol, datetime.datetime.strftime(d, '%Y-%m-%d'), ddd.shape[0]))
            if os.path.exists(p):
                continue
            if ddd.shape[0] > 1:
                day1S_df = ddd.resample('1S').last()
                day1S_df = day1S_df.interpolate()
                day1S_df = day1S_df.dropna()
                
                dfs_x = []
                dfs_y = []
                last_prices = []
                for (start, end) in between_times:
                    # X
                    _df = day1S_df.between_time(start, end)

                    if _df.shape[0] > 100:
                        ix = _df.index.values

                        df_x = _prepare_data(_df.last_price.values, start_ix) # len - 900 + 1
                        df_y = np.roll(df_x, -start_ix, axis=0)[:-start_ix,:next_tick_sec] # len - 900 + 1 - 900
                        df_x = df_x[:-start_ix]
                        
                        df_x = df_x - df_x.mean(axis=1, keepdims=True)

        #                 y = (df_y[:, 0] - df_y[:, -1]) / (df_y.max(axis=1) - df_y.min(axis=1))
                        df_y = (df_y[:, -1] - df_y[:, 0]) / df_y[:, 0]
                        df_y[np.isinf(df_y) | np.isneginf(df_y)] = 0
                        df_y[np.isnan(df_y)] = 0
                    
                        y_c = df_y.copy()
                        y_c[(y_c <= -0.0009)] = y_c[(y_c < -0.0009)] = -4
                        y_c[(y_c > -0.0009) & (y_c <= -0.0006)] = y_c[(y_c > -0.0009) * (y_c <= -0.0006)] = -3
                        y_c[(y_c > -0.0006) & (y_c <= -0.0003)] = y_c[(y_c > -0.0006) * (y_c <= -0.0003)] = -2
                        y_c[(y_c > -0.0003) & (y_c <= -0.0001)] = y_c[(y_c > -0.0003) * (y_c <= -0.0001)] = -1
                        
                        y_c[(y_c >= 0.0009)] = y_c[(y_c >= 0.0009)] = 4
                        y_c[(y_c >= 0.0006) & (y_c < 0.0009)] = y_c[(y_c >= 0.0006) & (y_c < 0.0009)] = 3
                        y_c[(y_c >= 0.0003) & (y_c < 0.0006)] = y_c[(y_c >= 0.0003) & (y_c < 0.0006)] = 2
                        y_c[(y_c >= 0.0001) & (y_c < 0.0003)] = y_c[(y_c >= 0.0001) & (y_c < 0.0003)] = 1
                        
                        y_c[(y_c > -0.0001) & (y_c < 0.0001)] = y_c[(y_c > -0.0001) & (y_c < 0.0001)] = 0
                        
                        df_y = np.column_stack([df_y, y_c])
                        
            #             y[df_y[:, 0] > df_y[:, -1]] = y[df_y[:, 0] > df_y[:, -1]] * -1
            #             if y.sum() > 0 and np.abs(y.mean()) != 1:
                        
#                         print(ix.shape)
#                         print(df_x.shape)
#                         print(df_y.shape)
                    
                        df_x = pd.DataFrame(df_x, index=np.roll(ix, -start_ix)[:-start_ix-start_ix+1]) # 前 start_ix 个数据的结束时间
                        df_y = pd.DataFrame(df_y, index=np.roll(ix, -start_ix-next_tick_sec)[:-start_ix-start_ix+1]) # 预测后 next_tick_sec 结束时间

                        dfs_x.append(df_x) # .iloc[:-start_ix])
                        dfs_y.append(df_y)
                        last_prices.append(_df.last_price)
                
                if len(dfs_x) > 0:
                    df_x = pd.concat(dfs_x)
                    df_y = pd.concat(dfs_y)
                    df_last_price = pd.concat(last_prices)


                    store = pd.HDFStore(p, 'w', complevel=7)

                    store.append('X', df_x)
                    store.append('y', df_y)
                    store.append('last_price', df_last_price)

                    store.flush()
                    store.close()
#                 print(exchange, symbol, d)
            
#     return #dfs_x, dfs_y, last_prices

In [ ]:
for df in dfs:
    exchange = 'SHFE'
    symbol = 'rb'
    prepare_data(df[['last_price', 'volume', 'amount']], exchange, symbol)

In [ ]:
store = pd.HDFStore(os.path.join(SAVE_PATH, 'SHFE.rb.2022-12-06.40848.h5'))
df_x = store['X']
df_y = store['y']

In [ ]:
df_y.values[:,1]

### 均值 螺纹主连

In [ ]:
# SAVE_PATH = 'D:/option/l8/l8_1S_1T_mean'
# SAVE_PATH = 'D:/option/l8/l8_10S_1T_mean'
# SAVE_PATH = 'D:/option/l8/l8_30S_1T_mean'
# SAVE_PATH = 'D:/option/l8/l8_30S_5T_mean'
# SAVE_PATH = 'D:/option/l8/l8_60S_10T_mean'
# SAVE_PATH = 'D:/option/l8/l8_30S_7T_mean'
SAVE_PATH = 'D:/option/l8/l8_30S_15T_mean'

In [ ]:
window_size = 32

In [ ]:
class Normalizer():
    def __init__(self):
        self.mu = None
        self.sd = None

    def fit_transform(self, x):
        self.mu = np.mean(x, axis=(0), keepdims=True)
        self.sd = np.std(x, axis=(0), keepdims=True)
        normalized_x = (x - self.mu)/self.sd
        return normalized_x

    def inverse_transform(self, x):
        return (x*self.sd) + self.mu

# normalize
# scaler = Normalizer()


In [ ]:
def prepare_data(df, exchange, symbol):
    
    between_times = [['09:00', '10:15'], ['10:30', '11:30'], ['13:30', '15:00'], ['21:00', '23:00']]
    
    def _prepare_data(x, window_size):
        n_row = x.shape[0] - window_size + 1
        output = np.lib.stride_tricks.as_strided(x, shape=(n_row, window_size), strides=(x.strides[0],x.strides[0]))
#         return output[:-1], output[-1]
        return output[:-1]
    
    with tqdm.tqdm(np.unique(df.index.date)) as bar:
        bar.set_description('%s %s' % (exchange, symbol))
        for d in bar:
            ddd = df[df.index.date == d]
            p = os.path.join(SAVE_PATH, '%s.%s.%s.%s.h5' % (exchange, symbol, datetime.datetime.strftime(d, '%Y-%m-%d'), ddd.shape[0]))
            if os.path.exists(p):
                continue
            if ddd.shape[0] > 1:
                day1S_df = ddd.resample('30S').last()
                day1S_df = day1S_df.interpolate()
                day1S_df = day1S_df.dropna()
                
                dfs_x = []
                dfs_y = []
                last_prices = []
                
                for (start, end) in between_times:
                    # X
                    _df = day1S_df.between_time(start, end).dropna()

                    if _df.shape[0] > 10:
#                         ix = _df.index.values
                        df_x = _df.last_price.rolling('15T').mean()
#                         df_x = normalize(df_x.diff(1).fillna(0).values.reshape(-1, 1), norm='l2')
                        last_price = _df.last_price[window_size:]
                        scaler = Normalizer()
                        df_x = scaler.fit_transform(df_x.to_numpy())
                        X = _prepare_data(df_x, window_size=window_size)
                        y = pd.DataFrame(df_x[window_size:], index=last_price.index, columns=['y'])
                        
                        y['mu'] = scaler.mu[0]
                        y['std'] = scaler.sd[0]
                        
                        dfs_x.append(pd.DataFrame(X, index=last_price.index)) # .iloc[:-start_ix])
                        dfs_y.append(y)
                        last_prices.append(pd.DataFrame(last_price.values, index=last_price.index, columns=['last_price']))
                                                    
                    if len(dfs_x) > 0:
                        df_x = pd.concat(dfs_x)
                        df_y = pd.concat(dfs_y)
                        df_last_price = pd.concat(last_prices)

                        store = pd.HDFStore(p, 'w', complevel=7)

                        store.append('X', df_x)
                        store.append('y', df_y)
                        store.append('last_price', df_last_price)

                        store.flush()
                        store.close()
            

In [ ]:
for df in dfs:
    exchange = 'SHFE'
    symbol = 'rb'
    prepare_data(df[['last_price', 'volume', 'amount']], exchange, symbol)
    


In [ ]:
def prepare_data_mu_std(df, exchange, symbol):
    
    def _prepare_data(x, window_size):
        n_row = x.shape[0] - window_size + 1
        output = np.lib.stride_tricks.as_strided(x, shape=(n_row, window_size), strides=(x.strides[0],x.strides[0]))
#         return output[:-1], output[-1]
        return output[:-1]
    
    mu_day_df = df.groupby([df.index.date]).mean()
    ix = mu_day_df.index
    
    scaler = Normalizer()
    mu_df = scaler.fit_transform(mu_day_df['mu'].to_numpy())
    
    mu_X = pd.DataFrame(_prepare_data(mu_df, window_size), index=ix[window_size:])
    mu_y = pd.DataFrame(mu_df[window_size:], index=ix[window_size:], columns=['y'])
    mu_y['mu'] = scaler.mu[0]
    mu_y['std'] = scaler.sd[0]
    
    store = pd.HDFStore(os.path.join('D:/option', '%s_%s_mu_day.h5' % (exchange, symbol)), 'w', complevel=7)
    store.append('X', mu_X)
    store.append('y', mu_y)
    store.flush()
    store.close()

In [ ]:
mu_stds = []

In [ ]:
for f in os.listdir(SAVE_PATH):
    print(f)
    store = pd.HDFStore(os.path.join(SAVE_PATH, f), 'r')
    y = store['y']
    store.close()
    
    mu_stds.append(y[['mu','std']].drop_duplicates(subset=['mu','std']))
    

In [ ]:
mu_std = pd.concat(mu_stds)

In [ ]:
prepare_data_mu_std(mu_std, 'SHFE', 'rb')

In [ ]:
%matplotlib qt

In [ ]:
plt.plot(mu_std['mu'])

In [ ]:
mu_df = mu_std.groupby([mu_std.index.date]).mean()

In [ ]:
mu_std

In [ ]:
plt.plot(mu_df['mu'])
plt.plot(mu_std['mu'])

### 均值 螺纹主连 Autoencoder

In [12]:
device = 0

In [13]:
# class Autoencoder(nn.Module):
#     def __init__(self): # 1 1
#         super(Autoencoder, self).__init__()
        
#         self.encoder = nn.Sequential(
#             nn.Linear(1, 128),
#             nn.BatchNorm1d(128),
#             nn.Sigmoid(),
#             nn.Linear(128, 64),
#             nn.Sigmoid(),
#             nn.Linear(64, 32),
#             nn.Sigmoid(),
#             nn.Linear(32, 1),
#         )
        
#         self.decoder = nn.Sequential(
#             nn.Linear(1, 32),
#             nn.Sigmoid(),
#             nn.Linear(32, 64),
#             nn.Sigmoid(),
#             nn.Linear(64, 128),
#             nn.Sigmoid(),
#             nn.BatchNorm1d(128),
#             nn.Linear(128, 1),
#         )
        
#     def forward(self, x):
#         encoded = self.encoder(x)
#         return self.decoder(encoded), encoded 

In [14]:
# model = Autoencoder().to(device)

In [15]:
# model.load_state_dict(torch.load('D:/option/models/SHFE.rb_encoder_256.pt'))

In [16]:
# model.eval()
# model.training

In [18]:
store = pd.HDFStore('D:/option/SHFE.rb.1S.h5', 'r')
df = store['rb']
store.close()

In [19]:
# class Normalizer():
#     def __init__(self):
#         self.model = Autoencoder().to(device)
#         self.model.load_state_dict(torch.load('D:/option/models/SHFE.rb_encoder.pt'))

#     def fit_transform(self, x):
#         return self.model.encoder(torch.tensor(x, dtype=torch.float32, device=0).reshape([-1, 1])).cpu().detach().numpy()

#     def inverse_transform(self, x):
#         return self.model.decoder(torch.tensor(x, dtype=torch.float32, device=0).reshape([-1, 1])).cpu().detach().numpy()

# scaler = Normalizer('D:/option/models/SHFE.rb_encoder_bn.pt', 0)

In [20]:
# scaler.model.training

In [21]:
window_size = 32
rule = '30S'
window = '15T'

In [22]:
# SAVE_PATH = 'D:/option/l8_autoencoder/1S_1T_mean'
# SAVE_PATH = 'D:/option/l8_autoencoder/10S_1T_mean'
# SAVE_PATH = 'D:/option/l8_autoencoder/30S_1T_mean'
# SAVE_PATH = 'D:/option/l8_autoencoder/30S_5T_mean'
# SAVE_PATH = 'D:/option/l8_autoencoder/60S_10T_mean'
# SAVE_PATH = 'D:/option/l8_autoencoder/30S_7T_mean'
# SAVE_PATH = 'D:/option/l8_autoencoder/30S_15T_mean'

# SAVE_PATH = os.path.join('D:/option/l8_autoencoder', '%s_%s_mean' % (rule, window))

SAVE_PATH = os.path.join('D:/option/master/mean', '%s_%s_mean' % (rule, window))


In [23]:
def prepare_data(df, exchange, symbol, window_size, rule, window, save_path):
    
    between_times = [['09:00', '10:15'], ['10:30', '11:30'], ['13:30', '15:00'], ['21:00', '23:00']]
    
    def _prepare_data(x, window_size):
        n_row = x.shape[0] - window_size + 1
        output = np.lib.stride_tricks.as_strided(x, shape=(n_row, window_size), strides=(x.strides[0],x.strides[0]))
#         return output[:-1], output[-1]
        return output[:-1]
    
    with tqdm.tqdm(np.unique(df.index.date)) as bar:
        bar.set_description('%s %s' % (exchange, symbol))
        for d in bar:
            ddd = df[df.index.date == d]
            p = os.path.join(save_path, '%s.%s.%s.%s.%s.%s.h5' % (exchange, symbol, datetime.datetime.strftime(d, '%Y-%m-%d'), rule, window, ddd.shape[0]))
            if os.path.exists(p):
                continue
            if ddd.shape[0] > 1:
                day1S_df = ddd.resample(rule).last()
                day1S_df = day1S_df.interpolate()
                day1S_df = day1S_df.dropna()
                
                dfs_x = []
                dfs_y = []
                last_prices = []
                
                for (start, end) in between_times:
                    # X
                    _df = day1S_df.between_time(start, end).dropna()

                    if _df.shape[0] > 10:
                        df_x = _df.last_price.rolling(window).mean()
                        last_price = _df.last_price[window_size:]
                        
#                         df_x = scaler.fit_transform(df_x.to_numpy())
                        df_x = df_x.to_numpy()
                        
                        X = _prepare_data(df_x, window_size=window_size)
                        y = pd.DataFrame(df_x[window_size:], index=last_price.index, columns=['y'])
                        
                        
                        dfs_x.append(pd.DataFrame(X, index=last_price.index)) # .iloc[:-start_ix])
                        dfs_y.append(y)
                        last_prices.append(pd.DataFrame(last_price.values, index=last_price.index, columns=['last_price']))
                                                    
                    if len(dfs_x) > 0:
                        df_x = pd.concat(dfs_x)
                        df_y = pd.concat(dfs_y)
                        df_last_price = pd.concat(last_prices)

                        store = pd.HDFStore(p, 'w', complevel=7)

                        store.append('X', df_x)
                        store.append('y', df_y)
                        store.append('last_price', df_last_price)

                        store.flush()
                        store.close()
            

In [21]:
prepare_data(df[['last_price', 'volume', 'amount']], 'SHFE', 'rb', window_size, rule, window, SAVE_PATH)

SHFE rb: 100%|███████████████████████████████| 533/533 [21:05<00:00,  2.37s/it]


In [21]:
%matplotlib qt

In [23]:
plt.plot(y1, label='y1')
plt.plot(y2.y, label='y2')
plt.legend()
plt.show()

In [13]:
d = df.loc['2023-05-04'].last_price.diff(1)

In [14]:
plt.plot(d.values)

### diff 螺纹主连

In [5]:
store = pd.HDFStore('D:/option/SHFE.rb.1S.h5', 'r')
df = store['rb']
store.close()

In [6]:
window_size = 32
rule = '30S'
window = '15T'

In [7]:
# SAVE_PATH = 'D:/option/l8_autoencoder/1S_1T_mean'
# SAVE_PATH = 'D:/option/l8_autoencoder/10S_1T_mean'
# SAVE_PATH = 'D:/option/l8_autoencoder/30S_1T_mean'
# SAVE_PATH = 'D:/option/l8_autoencoder/30S_5T_mean'
# SAVE_PATH = 'D:/option/l8_autoencoder/60S_10T_mean'
# SAVE_PATH = 'D:/option/l8_autoencoder/30S_7T_mean'
# SAVE_PATH = 'D:/option/l8_autoencoder/30S_15T_mean'

# SAVE_PATH = os.path.join('D:/option/l8_autoencoder', '%s_%s_mean' % (rule, window))

SAVE_PATH = os.path.join('D:/option/master/diff', '%s_%s' % (rule, window))

In [8]:
def prepare_data(df, exchange, symbol, window_size, rule, window, start_index, save_path):
    
    between_times = [['09:00', '10:15'], ['10:30', '11:30'], ['13:30', '15:00'], ['21:00', '23:00']]
    
    def _prepare_data(x, window_size):
        n_row = x.shape[0] - window_size + 1
        output = np.lib.stride_tricks.as_strided(x, shape=(n_row, window_size), strides=(x.strides[0],x.strides[0]))
        return output[:-1]
    
    with tqdm.tqdm(np.unique(df.index.date)) as bar:
        bar.set_description('%s %s' % (exchange, symbol))
        for d in bar:
            ddd = df[df.index.date == d]
            p = os.path.join(save_path, '%s.%s.%s.%s.%s.%s.h5' % (exchange, symbol, datetime.datetime.strftime(d, '%Y-%m-%d'), rule, window, ddd.shape[0]))
            if os.path.exists(p):
                continue
            if ddd.shape[0] > 1:
                day1S_df = ddd.resample(rule).last()
                day1S_df = day1S_df.interpolate()
                day1S_df = day1S_df.dropna()
                
                dfs_x = []
                dfs_y = []
                last_prices = []
                
                for (start, end) in between_times:
                    # X
                    _df = day1S_df.between_time(start, end).dropna()

                    if _df.shape[0] > 10:
                        df_x = _df.last_price.rolling(window).mean().diff(1)[start_index:]
                        last_price = _df.last_price[start_index:][window_size:]
                        
#                         df_x = scaler.fit_transform(df_x.to_numpy())
                        df_x = df_x.to_numpy()
                        
                        X = _prepare_data(df_x, window_size=window_size)
                        y = pd.DataFrame(df_x[window_size:], index=last_price.index, columns=['y'])
                        
                        
                        dfs_x.append(pd.DataFrame(X, index=last_price.index)) # .iloc[:-start_ix])
                        dfs_y.append(y)
                        last_prices.append(pd.DataFrame(last_price.values, index=last_price.index, columns=['last_price']))
                                                    
                    if len(dfs_x) > 0:
                        df_x = pd.concat(dfs_x)
                        df_y = pd.concat(dfs_y)
                        df_last_price = pd.concat(last_prices)

                        store = pd.HDFStore(p, 'w', complevel=7)

                        store.append('X', df_x)
                        store.append('y', df_y)
                        store.append('last_price', df_last_price)

                        store.flush()
                        store.close()
            

In [9]:
# prepare_data(df[['last_price', 'volume', 'amount']], 'SHFE', 'rb', window_size, rule, window, 60, SAVE_PATH) # 1S 1T
# prepare_data(df[['last_price', 'volume', 'amount']], 'SHFE', 'rb', window_size, rule, window, 6, SAVE_PATH)
# prepare_data(df[['last_price', 'volume', 'amount']], 'SHFE', 'rb', window_size, rule, window, 4, SAVE_PATH)
prepare_data(df[['last_price', 'volume', 'amount']], 'SHFE', 'rb', window_size, rule, window, 2, SAVE_PATH)

SHFE rb: 100%|███████████████████████████████| 530/530 [21:24<00:00,  2.42s/it]


In [ ]:
dd = df.loc['2023-05-04']

In [58]:
ddd = dd.between_time('09:00', '10:15')

In [60]:
dddd = ddd.resample(rule).last()

In [61]:
ddddd = dddd.diff(1).last_price.fillna(0)

In [64]:
ddddd[12:]

datetime
2023-05-04 09:01:00   -1.0
2023-05-04 09:01:05   -4.0
2023-05-04 09:01:10    2.0
2023-05-04 09:01:15   -2.0
2023-05-04 09:01:20   -2.0
                      ... 
2023-05-04 10:14:40    0.0
2023-05-04 10:14:45   -1.0
2023-05-04 10:14:50    0.0
2023-05-04 10:14:55    0.0
2023-05-04 10:15:00    1.0
Freq: 5S, Name: last_price, Length: 889, dtype: float64

In [67]:
ddddd.rolling(window).mean()[12:]

datetime
2023-05-04 09:01:00    0.083333
2023-05-04 09:01:05   -0.250000
2023-05-04 09:01:10   -0.333333
2023-05-04 09:01:15   -0.250000
2023-05-04 09:01:20   -0.416667
                         ...   
2023-05-04 10:14:40    0.083333
2023-05-04 10:14:45    0.083333
2023-05-04 10:14:50    0.083333
2023-05-04 10:14:55    0.083333
2023-05-04 10:15:00    0.166667
Freq: 5S, Name: last_price, Length: 889, dtype: float64

In [65]:
plt.plot(ddddd.rolling(window).mean().values)

In [12]:
plt.plot(ddd.diff(1).last_price)

In [24]:
plt.plot(ddd.last_price.rolling(window).mean().diff(1))

In [27]:
sn.displot(ddd.last_price.rolling(window).mean().diff(1), kde=True)

In [26]:
sn.displot(ddd.last_price.rolling(window).mean(), kde=True)

In [39]:
plt.plot(ddd.last_price.rolling('1T').mean().diff(1))

In [37]:
plt.plot(ddd.last_price.rolling(window).mean()[60:])

In [35]:
ddd.last_price.rolling('3T').mean()

datetime
2023-05-04 09:00:00    3645.000000
2023-05-04 09:00:01    3641.500000
2023-05-04 09:00:02    3640.333333
2023-05-04 09:00:03    3640.500000
2023-05-04 09:00:04    3640.000000
                          ...     
2023-05-04 10:14:56    3668.572222
2023-05-04 10:14:57    3668.594444
2023-05-04 10:14:58    3668.622222
2023-05-04 10:14:59    3668.638889
2023-05-04 10:15:00    3668.655556
Name: last_price, Length: 4501, dtype: float64

In [44]:
sn.displot(ddd.last_price.rolling('1T').mean().diff(1)[60:], kde=True)

In [47]:
df_x = ddd.last_price.rolling('1T').mean().diff(1)[60:]

In [48]:
df_x

datetime
2023-05-04 09:01:00   -0.083333
2023-05-04 09:01:01   -0.016667
2023-05-04 09:01:02   -0.016667
2023-05-04 09:01:03   -0.066667
2023-05-04 09:01:04    0.016667
                         ...   
2023-05-04 10:14:56    0.033333
2023-05-04 10:14:57    0.033333
2023-05-04 10:14:58    0.050000
2023-05-04 10:14:59    0.016667
2023-05-04 10:15:00    0.050000
Name: last_price, Length: 4441, dtype: float64

### macd boll

In [5]:
store = pd.HDFStore('D:/option/SHFE.rb.1S.h5', 'r')
df = store['rb']
store.close()

In [4]:
window_size = 32
rule = '1S'
window = '5T'

In [5]:
SAVE_PATH = os.path.join('D:/option/master/ind', '%s_%s' % (rule, window))

In [ ]:
def prepare_data(df, exchange, symbol, window_size, rule, window, start_index, save_path):
    
    between_times = [['09:00', '10:15'], ['10:30', '11:30'], ['13:30', '15:00'], ['21:00', '23:00']]
    
    def _prepare_data(x, window_size):
        n_row = x.shape[0] - window_size + 1
        output = np.lib.stride_tricks.as_strided(x, shape=(n_row, window_size), strides=(x.strides[0],x.strides[0]))
        return output[:-1]
    
    with tqdm.tqdm(np.unique(df.index.date)) as bar:
        bar.set_description('%s %s' % (exchange, symbol))
        for d in bar:
            ddd = df[df.index.date == d]
            p = os.path.join(save_path, '%s.%s.%s.%s.%s.%s.h5' % (exchange, symbol, datetime.datetime.strftime(d, '%Y-%m-%d'), rule, window, ddd.shape[0]))
            if os.path.exists(p):
                continue
            if ddd.shape[0] > 1:
                day1S_df = ddd.resample(rule).last()
                day1S_df = day1S_df.interpolate()
                day1S_df = day1S_df.dropna()
                
                dfs_x = []
                dfs_y = []
                last_prices = []
                
                for (start, end) in between_times:
                    # X
                    _df = day1S_df.between_time(start, end).dropna()

                    if _df.shape[0] > 10:
                        df_x = _df.last_price.rolling(window).mean().diff(1)[start_index:]
                        last_price = _df.last_price[start_index:][window_size:]
                        
#                         df_x = scaler.fit_transform(df_x.to_numpy())
                        df_x = df_x.to_numpy()
                        
                        X = _prepare_data(df_x, window_size=window_size)
                        y = pd.DataFrame(df_x[window_size:], index=last_price.index, columns=['y'])
                        
                        
                        dfs_x.append(pd.DataFrame(X, index=last_price.index)) # .iloc[:-start_ix])
                        dfs_y.append(y)
                        last_prices.append(pd.DataFrame(last_price.values, index=last_price.index, columns=['last_price']))
                                                    
                    if len(dfs_x) > 0:
                        df_x = pd.concat(dfs_x)
                        df_y = pd.concat(dfs_y)
                        df_last_price = pd.concat(last_prices)

                        store = pd.HDFStore(p, 'w', complevel=7)

                        store.append('X', df_x)
                        store.append('y', df_y)
                        store.append('last_price', df_last_price)

                        store.flush()
                        store.close()
            

In [10]:
d = df.last_price.loc['2021-04-06'].between_time('09:00', '10:15').interpolate()

In [12]:
d.diff(1).plot()

<AxesSubplot:xlabel='datetime'>

In [13]:
df.datetime.unique()

array(['2021-04-06T09:00:00.500001000', '2021-04-06T09:00:01.500000000',
       '2021-04-06T09:00:02.500000000', ...,
       '2023-06-09T22:59:58.500000000', '2023-06-09T22:59:59.500000000',
       '2023-06-09T23:00:00.500000000'], dtype='datetime64[ns]')

### 多维

In [5]:
store = pd.HDFStore('D:/option/SHFE.rb.h5', 'r')
df = store['rb']
store.close()

In [7]:
window_size = 32
rule = '30S'
window = '15T'
start_index = int(60 / 30)

In [8]:
SAVE_PATH = os.path.join('D:/option/master/dims/diff', '%s_%s' % (rule, window))
if not os.path.exists(SAVE_PATH): os.makedirs(SAVE_PATH)

In [9]:
def prepare_data(df, exchange, symbol, window_size, rule, window, start_index, save_path):
    
    between_times = [['09:00', '10:15'], ['10:30', '11:30'], ['13:30', '15:00'], ['21:00', '23:00']]
    
    def _prepare_data(x, window_size):
        n_row = x.shape[0] - window_size + 1
        output = np.lib.stride_tricks.as_strided(x, shape=(n_row, window_size), strides=(x.strides[0],x.strides[0]))
        return output[:-1]
    
    with tqdm.tqdm(np.unique(df.index.date)) as bar:
        bar.set_description('%s %s' % (exchange, symbol))
        for d in bar:
            day_df = df[df.index.date == d]
            p = os.path.join(save_path, '%s.%s.%s.%s.%s.%s.h5' % (exchange, symbol, datetime.datetime.strftime(d, '%Y-%m-%d'), rule, window, day_df.shape[0]))
            if os.path.exists(p):
                continue
            if day_df.shape[0] > 1:
                day_df_price = day_df.last_price.resample(rule).last()
#                 day_df_price = day_df_price.interpolate()
#                 day_df_price = day_df_price.dropna()
                
                day_df_volume = day_df.volume.resample(rule).mean()
            
                day_df_amount = day_df.amount.resample(rule).mean()
                
                train_last_prices = []
                train_volumes = []
                train_amounts = []
                
                train_ys = []
                
                last_prices = []
                
                for (start, end) in between_times:
                    # X
                    _df_price = day_df_price.between_time(start, end)
                    _df_volume = day_df_volume.between_time(start, end)
                    _df_amount = day_df_amount.between_time(start, end)

                    if _df_price.shape[0] > 10:
                        X_price_roll = _df_price.rolling(window).mean().diff(1)[start_index:]
                        X_volume_roll = _df_volume.rolling(window).mean().diff(1)[start_index:]
                        X_amount_roll = _df_amount.rolling(window).mean().diff(1)[start_index:]
                        
                        last_price = _df_price.iloc[start_index:].iloc[window_size:]
                        
                        X_price = _prepare_data(X_price_roll.to_numpy(), window_size=window_size)
                        X_volume = _prepare_data(X_volume_roll.to_numpy(), window_size=window_size)
                        X_amount = _prepare_data(X_amount_roll.to_numpy(), window_size=window_size)
                        
                        train_ys.append(pd.DataFrame(X_price_roll[window_size:].values, index=last_price.index, columns=['y']))
                        
                        train_last_prices.append(pd.DataFrame(X_price, index=last_price.index))
                        train_volumes.append(pd.DataFrame(X_volume, index=last_price.index))
                        train_amounts.append(pd.DataFrame(X_amount, index=last_price.index))
                        
                        last_prices.append(pd.DataFrame(last_price.values, index=last_price.index, columns=['price']))
                                                    
                    if len(train_last_prices) > 0:
                        train_last_price = pd.concat(train_last_prices)
                        train_volume = pd.concat(train_volumes)
                        train_amount = pd.concat(train_amounts)
                        
                        df_last_price = pd.concat(last_prices)
                        
                        train_y = pd.concat(train_ys)

                        store = pd.HDFStore(p, 'w', complevel=7)

                        store.append('X_price', train_last_price)
                        store.append('X_volume', train_volume)
                        store.append('X_amount', train_amount)
                        
                        store.append('price', df_last_price)
                        
                        store.append('y', train_y)
                        
                        
                        store.flush()
                        store.close()
            

In [10]:
prepare_data(df[['last_price', 'volume', 'amount']], 'SHFE', 'rb', window_size, rule, window, start_index, SAVE_PATH)

SHFE rb: 100%|███████████████████████████████| 532/532 [41:00<00:00,  4.63s/it]


In [11]:
a = os.path.join('D:/option/master/dims/diff/30S_15T', 'SHFE.rb.2021-04-06.30S.15T.41253.h5')
store = pd.HDFStore(a, 'r')
y = store['y']
store.close()

In [16]:
plt.plot(y.values, label='real', color='r', marker='.')